# Proyecto Bimestral: Implementación de un Sistema RAG (Retrieval-Augmented Generation)
## 1. Introducción
Este proyecto consiste en diseñar e implementar un sistema RAG (Retrieval-Augmented Generation). Se busca combinar técnicas de Recuperación de Información (RI) con modelos de generación de texto, permitiendo que un modelo genere respuestas a partir de documentos relevantes recuperados desde  un corpus.

## 2. Objetivo del Proyecto
Desarrollar un sistema RAG que:
1. Recupere documentos relevantes a partir de una consulta del usuario utilizando técnicas de RI.
2. Genere respuestas basadas en los documentos recuperados utilizando un modelo de lenguaje avanzado.

## 3. Fases del Proyecto


#### Instalación de bibliotecas para Recuperación de Información

Este bloque de código instala las bibliotecas necesarias para trabajar con tareas relacionadas con la Recuperación de Información, búsqueda vectorial y procesamiento de lenguaje natural. A continuación, se describe la función de cada comando:

1. **`pip install chromadb`**  
   Instala la biblioteca **ChromaDB**, una base de datos orientada a la búsqueda y gestión de embeddings. ChromaDB es utilizada para almacenar y consultar representaciones vectoriales de datos de manera eficiente.

2. **`pip install --upgrade chromadb`**  
   Actualiza **ChromaDB** a la versión más reciente para asegurarse de disponer de las últimas características y correcciones de errores.

3. **`!pip install chroma-migrate`**  
   Instala la herramienta **chroma-migrate**, que permite la migración de datos y la gestión de versiones en bases de datos construidas con ChromaDB.

4. **`pip install sentence-transformers`**  
   Instala **Sentence Transformers**, una biblioteca utilizada para generar embeddings de texto. Esta biblioteca es esencial para tareas de búsqueda semántica y comparación de textos, facilitando la conversión de oraciones a representaciones vectoriales.

5. **`!pip install openai`**  
   Instala la biblioteca **OpenAI**, que permite la interacción con la API de OpenAI para utilizar modelos de lenguaje como GPT en tareas de procesamiento de texto.

### Observaciones:
- El prefijo `!` se usa para ejecutar comandos de terminal directamente desde celdas de código en un entorno Jupyter Notebook.  
- Se recomienda reiniciar el kernel después de instalar o actualizar las bibliotecas para evitar problemas de compatibilidad.

###  3.1. Importación de bibliotecas para Recuperación de Información

In [ ]:
import pandas as pd  # Para manejar datos tabulares.
from sentence_transformers import SentenceTransformer  # Para generar embeddings de texto.
import chromadb  # Base de datos para almacenar y consultar embeddings.
from chromadb.config import Settings  # Configuración de ChromaDB.
from openai import OpenAI  # Para interactuar con modelos de OpenAI.

###  3.2. Generación de Embeddings a partir de texto

Este bloque de código se enfoca en cargar un modelo preentrenado para generar embeddings de texto y en leer un archivo CSV que contiene datos a ser procesados. A continuación, se explican las acciones realizadas:

In [3]:
# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Leer el archivo CSV
file_path = "./Data/Entrevista_LuisaGonzales_JuanCueva.csv"
data = pd.read_csv(file_path, sep=",")  # Asegúrate de usar el separador adecuado (tabulación en este caso)

In [4]:
data

,ID,Candidato,Temas,Descripción,Entrevista
0,LG1,Luisa Gonzales,justicia social propuesta corrupción,entrevista relevante entender propuesta visión...,buen día lenin saludo escuchar momento revoluc...
1,LG2,Luisa Gonzales,crisis eléctrico relación internacional acción,entrevista ofrecer visión claro prioridad estr...,mucho gracia buen día ecuatoriano mirar moment...
2,LG3,Luisa Gonzales,venezuela rafael correo persecución,gonzález defender independencia capacidad lide...,querido fernando necesitar consultar ninguno a...
3,JC1,Juan Cueva,propiedad intelectual panorama ecuatoriano,abordo tema clave relacionado protección gesti...,propiedad intelectual derecho cualquiera autor...
4,JC2,Juan Cueva,proyecto político experiencia corrupción,entrevista juan iván cueva candidato presidenc...,mucho gracia lenin invitación honor aquí prese...
5,JC3,Juan Cueva,colaboración sector desafío protección caso éx...,entrevista juan iván cueva compartir experienc...,estrategia considerar efectivo ver protección ...


### 3.3. Extracción de columnas, generación de embeddings y preparación de datos

Este bloque de código se encarga de extraer las columnas relevantes del DataFrame para su procesamiento, generar los embeddings a partir del texto de los documentos y organizar los metadatos asociados. Se detalla cada paso a continuación:


In [5]:
# Extraer las columnas de interés
documents = data['Entrevista'].tolist()  # El texto del documento
ids = data['ID'].tolist()  # Identificadores únicos
metadatas = data[['Candidato', 'Temas', 'Descripción']].to_dict(orient='records')  # Metadatos

In [6]:
# Generar los embeddings para los documentos
embeddings = model.encode(documents).tolist()

### 3.4. Inicialización de la base de datos vectorial y almacenamiento de documentos

Este bloque de código se enfoca en la creación de una base de datos vectorial persistente utilizando **ChromaDB**, la creación o carga de una colección dentro de esta base, y la adición de documentos junto con sus embeddings, metadatos e identificadores únicos. A continuación, se describe cada paso:


In [7]:
# Inicializar la base de datos vectorial con persistencia
client = chromadb.PersistentClient(
    path="./chroma_db"  # Carpeta donde se guardarán los datos
)

In [8]:
# Crear o cargar la colección
collection = client.get_or_create_collection("prueba_collection")

# Agregar los documentos, metadatos, IDs y embeddings a la colección
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embeddings=embeddings
)

print("Documentos agregados exitosamente a la colección.")

Insert of existing embedding ID: LG1
Insert of existing embedding ID: LG2
Insert of existing embedding ID: LG3
Insert of existing embedding ID: JC1
Insert of existing embedding ID: JC2
Insert of existing embedding ID: JC3
Add of existing embedding ID: LG1
Add of existing embedding ID: LG2
Add of existing embedding ID: LG3
Add of existing embedding ID: JC1
Add of existing embedding ID: JC2
Add of existing embedding ID: JC3


Documentos agregados exitosamente a la colección.


### 3.5. Realización de una consulta semántica en la base de datos vectorial

Este bloque de código realiza una consulta semántica en la base de datos vectorial utilizando el embedding de una pregunta en lenguaje natural. Se busca encontrar los documentos más relevantes basados en la similitud semántica con la consulta. A continuación, se explica cada paso:

In [9]:
# Realizar una consulta
query = "¿Cómo enfrentará luisa gonzales la crisis eléctrica?"  # Tu consulta en texto
query_embedding = model.encode([query]).tolist()  # Generar el embedding de la consulta

# Consultar los documentos más similares
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3  # Devuelve los 3 documentos más similares
)

# Imprimir los resultados
print("Resultados de la consulta:")
for result in results['documents']:
    print(result)

Resultados de la consulta:
['buen día lenin saludo escuchar momento revolución ciudadano proyecto patria buscar justicia social vida armonía paz hoy querer revivir ecuador agonizar falta empleo salud educación objetivo devolver esperanza alegría día mejor proyecto ahora encabezado mujer luisa gonzález junto diego borja representar sierra ecuatoriano país cambiar proyecto hoy vivir desesperanza tristeza violencia sumir opción empleo inseguridad disparar ecuador seguro hoy país violento sudamérica según human rights plan enfocado construir paz justicia social enfoque estructurado seguridad plan protege contemplar equipar fuerza público mejorar inteligencia trabajar manera coordinado fuerza armadas policía nacional además plan impulsa trabajarer área específico niño joven mujer cabezo hogar adulto mayor prioridad unir patria dividido rafael correo seguir proceso corte internacional demostrar caso persecución político cuanto jorge glas si ley tratado internacional determinar deber recibir 

#### Listado de colecciones y recuperación de datos almacenados

Este bloque de código se enfoca en listar las colecciones existentes en la base de datos vectorial, obtener una colección específica por su nombre y recuperar todos los datos almacenados en ella. Se detallan los pasos a continuación:


In [10]:
# Listar las colecciones existentes
collections = client.list_collections()

# Imprimir las colecciones
for collection in collections:
    print("Nombre de la colección:", collection.name)

Nombre de la colección: prueba_collection


In [11]:
# Obtener la colección por nombre
collection = client.get_collection("prueba_collection")

# Recuperar todos los datos (embeddings, IDs y metadatos)
datos = collection.get()

# Imprimir el contenido
print("IDs:", datos["ids"])
print("Embeddings:", datos["embeddings"])
print("Metadatos:", datos["metadatas"])

IDs: ['LG1', 'LG2', 'LG3', 'JC1', 'JC2', 'JC3']
Embeddings: None
Metadatos: [{'Candidato': 'Luisa Gonzales', 'Descripción': 'entrevista relevante entender propuesta visión político luisa gonzález contexto crisis económico social seguridad ecuador', 'Temas': 'justicia social propuesta corrupción'}, {'Candidato': 'Luisa Gonzales', 'Descripción': 'entrevista ofrecer visión claro prioridad estrategia luisa gonzález enfrentar desafío económico social político ecuador', 'Temas': 'crisis eléctrico relación internacional acción'}, {'Candidato': 'Luisa Gonzales', 'Descripción': 'gonzález defender independencia capacidad liderar criticar justicia ecuatoriano enfatizar necesidad unidad liderazgo superar crisis país además responder pregunta relación rafael correa corrupción visión política exterior', 'Temas': 'venezuela rafael correo persecución'}, {'Candidato': 'Juan Cueva', 'Descripción': 'abordo tema clave relacionado protección gestión derecho propiedad intelectual explicar concepto fundament

### 3.6. Configuración de la API de OpenAI y preparación del contexto para consultas a GPT

Este bloque de código configura el cliente para interactuar con la API de OpenAI y prepara el contexto basado en los documentos más relevantes recuperados de la base de datos vectorial. A continuación, se detalla cada paso:


In [ ]:
# Configuración de OpenAI API
cliente = OpenAI(
    api_key='XXXXXX-XXXXXX-XXXXXX'
)

In [16]:
# Preparar el contexto para el modelo GPT
context = "\n".join(results['documents'][0])  # Combinar los documentos más relevantes

### 3.7. Uso del modelo GPT para generar respuestas basadas en el contexto recuperado

Este bloque de código utiliza la **API de OpenAI** para generar una respuesta basada en el contexto recuperado de la base de datos vectorial y la consulta del usuario.


In [20]:
# Usar GPT para generar una respuesta basada en el contexto recuperado
response = cliente.chat.completions.create(
    model="gpt-4o-mini",  # Usar el modelo especificado
    messages=[
        {"role": "system", "content": "Eres un experto en recuperación de información que proporciona respuestas breves y concisas."},
        {"role": "user", "content": f"Usando el siguiente contexto, responde de manera breve y concisa a la pregunta: '{query}'\n\nContexto:\n{context}\n\nRespuesta:"}
    ]
)

In [21]:
response.choices[0].message

ChatCompletionMessage(content='Luisa González enfrentará la crisis eléctrica rehabilitando y operando al máximo las termoeléctricas existentes, desarrollando proyectos eólicos y fotovoltaicos, y eliminando gastos innecesarios asociados a la compra de energía. Su enfoque incluye fortalecer la infraestructura energética y garantizar una gestión eficiente.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)